In [20]:
import pandas as pd
import re
sample = ['http://1.1.1.1',
          'https://1.2.3.4',
          'http://10.1.2.3',
          None,
          'https://www.google.com']
# feel free to use a more precise regex if you want.
re_IPv4_url = re.compile('https?:\/\/([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3})')
df = pd.DataFrame(sample,columns=['referrer'])

Make a DataFrame with some mixed data:

In [21]:
df

,referrer
0,http://1.1.1.1
1,https://1.2.3.4
2,http://10.1.2.3
3,None
4,https://www.google.com


Extract referrers that contain IPv4 literals:

In [22]:
literal_ip_referrers = df[df.referrer.fillna('').str.contains(re_IPv4_url)].referrer.unique()
literal_ip_referrers

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


array(['http://1.1.1.1', 'https://1.2.3.4', 'http://10.1.2.3'],
      dtype=object)

Extract just the IP addresses:

In [19]:
ip_addrs = df.referrer.fillna('').str.extract(re_IPv4_url).dropna()
ip_addrs

,0
0,1.1.1.1
1,1.2.3.4
2,10.1.2.3
